In [5]:
import pandas as pd

In [141]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, WebDriverException

import time

options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

In [145]:

service = Service("/Users/robertvelez/.wdm/drivers/chromedriver/mac64/133.0.6943.98/chromedriver-mac-arm64/chromedriver")
driver = webdriver.Chrome(options=options)
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options)

In [93]:
leagues = {
    "2023-2024": {
        "Premier League": "https://www.whoscored.com/regions/252/tournaments/2/seasons/9618/stages/22076/playerstatistics/england-premier-league-2023-2024",
        "Bundesliga": "https://www.whoscored.com/regions/81/tournaments/3/seasons/9649/stages/22128/playerstatistics/germany-bundesliga-2023-2024",
        "La Liga": "https://www.whoscored.com/regions/206/tournaments/4/seasons/9682/stages/22176/playerstatistics/spain-laliga-2023-2024",
        "Serie A": "https://www.whoscored.com/regions/108/tournaments/5/seasons/9659/stages/22143/playerstatistics/italy-serie-a-2023-2024",
        "Ligue 1": "https://www.whoscored.com/regions/74/tournaments/22/seasons/9635/stages/22105/playerstatistics/france-ligue-1-2023-2024"
    },
    "2022-2023": {
        "Premier League": "https://www.whoscored.com/regions/252/tournaments/2/seasons/9075/stages/20934/playerstatistics/england-premier-league-2022-2023",
        "Bundesliga": "https://www.whoscored.com/regions/81/tournaments/3/seasons/9120/stages/21026/playerstatistics/germany-bundesliga-2022-2023",
        "La Liga": "https://www.whoscored.com/regions/206/tournaments/4/seasons/9149/stages/21073/playerstatistics/spain-laliga-2022-2023",
        "Serie A": "https://www.whoscored.com/regions/108/tournaments/5/seasons/9159/stages/21087/playerstatistics/italy-serie-a-2022-2023",
        "Ligue 1": "https://www.whoscored.com/regions/74/tournaments/22/seasons/9129/stages/21037/playerstatistics/france-ligue-1-2022-2023"
    },
    "2021-2022": {
        "Premier League": "https://www.whoscored.com/regions/252/tournaments/2/seasons/8618/stages/19793/playerstatistics/england-premier-league-2021-2022",
        "Bundesliga": "https://www.whoscored.com/regions/81/tournaments/3/seasons/8667/stages/19862/playerstatistics/germany-bundesliga-2021-2022",
        "La Liga": "https://www.whoscored.com/regions/206/tournaments/4/seasons/8681/stages/19895/playerstatistics/spain-laliga-2021-2022",
        "Serie A": "https://www.whoscored.com/regions/108/tournaments/5/seasons/8735/stages/19982/playerstatistics/italy-serie-a-2021-2022",
        "Ligue 1": "https://www.whoscored.com/regions/74/tournaments/22/seasons/8671/stages/19866/playerstatistics/france-ligue-1-2021-2022"
    }
}


def scrape_all_leagues(season, leagues):
    all_output_data = []

    for league_name, url in leagues.items():
        print(f"Scraping {league_name} for the {season} season...")

        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Click the "All Players" button
        all_players_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='apps']/dd[2]/a")))
        all_players_button.click()
        time.sleep(3)
        print(f"Clicked the 'All players' button for {season} season successfully!")

        output_data = []
        while True:
            try:
                # Wait for player table to load
                player_table = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='player-table-statistics-body']")))
                table_html = "<table>" + player_table.get_attribute('innerHTML') + "</table>"

                # Read HTML table with pandas
                df = pd.read_html(table_html)[0]

                # Extract Player Name and Rating
                player_data = df.iloc[:, [0, -1]]  # First column (Name) & last column (Rating)
                player_data.columns = ["Player Name", "Rating"]
                player_data["League"] = league_name
                player_data["Season"] = season  # Add Season column

                output_data.append(player_data)

                # Locate the "Next" button
                next_button = driver.find_element(By.LINK_TEXT, "next")

                # Check if the button is disabled
                if "disabled" in next_button.get_attribute("class"):
                    print(f"Last page reached for {league_name} ({season}).")
                    break

                # Click the "Next" button
                next_button.click()
                time.sleep(5)  # Allow time for next page to load

            except Exception as e:
                print(f"Error in {league_name} ({season}): {e}")
                break

        # Add the data for the current league to the all_output_data list
        all_output_data.extend(output_data)

    # Save all collected data for the given season to a single CSV file
    if all_output_data:
        final_df = pd.concat(all_output_data, ignore_index=True)
        final_df.to_csv(f"top5leagues_ratings_{season}.csv", mode='a', header=True, index=False)

# Scraping for each season
for season, league_urls in leagues.items():
    scrape_all_leagues(season, league_urls)

driver.quit()


Scraping Premier League for the 2023-2024 season...
Clicked the 'All players' button for 2023-2024 season successfully!


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Error in Premier League (2023-2024): Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=133.0.6943.98); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102e82bac cxxbridge1$str$ptr + 2724820
1   chromedriver                        0x0000000102e7b20c cxxbridge1$str$ptr + 2693684
2   chromedriver                        0x00000001029e1afc cxxbridge1$string$len + 93348
3   chromedriver                        0x00000001029e7808 cxxbridge1$string$len + 117168
4   chromedriver                        0x00000001029e98e0 cxxbridge1$string$len + 125576
5   chromedriver                        0x00000001029e9988 cxxbridge1$string$len + 125744
6   chromedriver                        0x0000000102a2e2d8 cxxbridge1$string$len + 406656
7   chromedriver                        0x0000000102a2

/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Clicked the 'All players' button for 2023-2024 season successfully!


...

Last page reached for La Liga (2021-2022).
Scraping Serie A for the 2021-2022 season...
Clicked the 'All players' button for 2021-2022 season successfully!


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Last page reached for Serie A (2021-2022).
Scraping Ligue 1 for the 2021-2022 season...
Clicked the 'All players' button for 2021-2022 season successfully!


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Last page reached for Ligue 1 (2021-2022).


In [146]:
failed_2023_2024 = { "2023-2024": {
        "Premier League": "https://www.whoscored.com/regions/252/tournaments/2/seasons/9618/stages/22076/playerstatistics/england-premier-league-2023-2024",
        "Bundesliga": "https://www.whoscored.com/regions/81/tournaments/3/seasons/9649/stages/22128/playerstatistics/germany-bundesliga-2023-2024"}
}

for season, league_urls in failed_2023_2024.items():
    scrape_all_leagues(season, league_urls)

driver.quit()

Scraping Premier League for the 2023-2024 season...
Clicked the 'All players' button for 2023-2024 season successfully!


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Last page reached for Premier League (2023-2024).
Scraping Bundesliga for the 2023-2024 season...
Clicked the 'All players' button for 2023-2024 season successfully!


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Last page reached for Bundesliga (2023-2024).


/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data["League"] = league_name
/var/folders/rj/gkzv_bs14qg22pdz7mcwyfww0000gn/T/ipykernel_13289/3570572440.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d